In [6]:
!pip install -U git+https://github.com/shahashka/SP-GIES --no-cache-dir

  Cloning https://github.com/shahashka/SP-GIES to /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-m4wt6hal
  Running command git clone --filter=blob:none --quiet https://github.com/shahashka/SP-GIES /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-m4wt6hal
  Resolved https://github.com/shahashka/SP-GIES to commit f1df54725af423aef3215b52cb441e809d126476
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-m4wt6hal/setup.py:9: SetuptoolsDeprecationWarning: As setuptools moves its configuration towards `pyproject.toml`,
      `setuptools.config.read_configuration` became deprecated.
      
      For the time being, you can use the `setuptools.config.setupcfg` module
      to acce

In [1]:
from sp_gies import utils
from sp_gies import sp_gies as sp
import sp_gies
sp_gies.__version__

ModuleNotFoundError: No module named 'networkx'

## Generate a random graph and random data.
Data is sampled from a linear Gaussian model (child nodes are weighted sums of parent nodes). Options for random graphs are scale_free, erdos_renyi, and small_world. Choose the number of nodes in the graphs (n), number of observational samples (nsamples) and number of interventional samples per node (iv_samples). p, k are parameters for the random graphs.

Arcs is a list of edge pairs e.g. [("G1","G2"), ("G2","G4")]

In [3]:
(arcs,nodes,bias,var), data = utils.get_random_graph_data("scale_free", n=10, nsamples=10000, iv_samples=10, p=0.5, k=2)

The dataset should have column names corresponding to the random variables names. Additionally a 'target' column specifies which intervention index was performed to obtain the sample. The target column indexes from 1 (for compatability with R). For example in the dataframe below, target=1 indicates that 'G1' was intervened to obtain the corresponding interventional sample. For observational samples the target value should be 0. 


In [4]:
print(data.head)

<bound method NDFrame.head of            G1        G2        G3        G4        G5        G6         G7  \
0   14.464795  4.278928 -0.770665  1.658923  1.135153 -0.494724   6.868821   
1   18.019292  6.487468  0.569230  3.519605  5.462579 -0.430048  13.539153   
2    9.352399  3.845686 -1.395816  1.980282  0.773106 -0.608937   5.710223   
3   10.553419  4.965106 -0.565652  2.184134  1.510322 -0.705889   7.432769   
4   18.561109  6.504644  0.223877  3.764172  3.001941 -0.478044  10.017195   
..        ...       ...       ...       ...       ...       ...        ...   
5   23.975807  7.728077  2.381381  3.797984  8.787509 -0.180321  16.719876   
6   21.516556  7.976141  0.292340  5.111815  6.127549 -0.619453  14.327381   
7   15.239631  4.946446  0.520864  2.631599  4.472130 -0.854605   9.754562   
8   15.107945  5.971126 -0.431315  3.494727  3.279704 -0.839910   9.501493   
9    3.661724  3.935389 -2.421017  1.799909  0.328601 -0.989725   4.502166   

           G8        G9       G10

# Run the SP-GIES algorithm on the dataset.
Runs the PC algorithm if pc=True and then the GIES algorithm on the output CPDAG.
Optionally provide an input skeleton, which should be a numpy array with number rows/columns corresponding to the number of nodes. The output adjacency matrix is saved in the output dir as sp_gies-adj_mat.csv.

For a 10 node graph this should take less than 1 second

In [5]:
adj_mat = sp.sp_gies(data, outdir="./", skel=None, pc=True)

TypeError: 'bool' object is not callable

# Evaluate the estimated graph.
Convert to networkx Digraph and run evaluation metrics. 

SHD is the L1 error between the graphs.

SID is the number of wrong interventional distributions. This method scales quadratically with the number of nodes. For large graphs it can take a while, so there is an option to turn off this evaluation with get_sid=False.

AUC is the error under the precision recall curve

In [10]:
G_true = utils.edge_to_dag(arcs)
G_sp_gies = utils.adj_to_dag(adj_mat, nodes)
print(utils.get_scores(["SP-GIES"], [G_sp_gies], G_true, get_sid=True))

NameError: name 'adj_mat' is not defined